In [1]:
import os
from osgeo import gdal

band_path = "../../data/images"
band_numbers = ['2', '3', '4', '5', '6', '7', '8', '8A', '11', '12']
band_date = [
    "_20220326-105856-076_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220405-105855-542_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220714-105903-040_L2A_T31TCJ_C_V3-0_FRE_B",
    "_20220922-105859-863_L2A_T31TCJ_C_V3-1_FRE_B",
    "_20221111-105858-090_L2A_T31TCJ_C_V3-1_FRE_B",
    "_20230219-105857-687_L2A_T31TCJ_C_V3-1_FRE_B"
]  # Liste des dates pour chaque image

# Générer les chemins complets pour chaque bande
band_files = [f"{band_path}/SENTINEL2B{date}{band_number}.tif" for date in band_date for band_number in band_numbers]

# Répertoire de sortie
out_path = "../results/data/img_pretraitees"
mask_file = os.path.join(out_path, "masque_foret.tif")  # Fichier de masque

# Paramètres de sortie
output_file = os.path.join(out_path, "Serie_temp_S2_allbands.tif")
projection = "EPSG:2154"
pixel_size = 10  # Résolution spatiale en mètres
nodata_value = 0
out_type = "UInt16"

In [2]:
# Fonction pour reprojeter et rééchantillonner une bande
def reproject_and_resample(band_file, output_file, pixel_size, projection):
    # Ouvrir le fichier de la bande comme un dataset GDAL
    band_ds = gdal.Open(band_file)
    if band_ds is None:
        raise Exception(f"Erreur lors de l'ouverture du fichier {band_file}")

    # Reprojection et rééchantillonnage
    gdal.Warp(
        output_file,
        band_ds,
        xRes=pixel_size,
        yRes=pixel_size,
        dstSRS=projection,
        dstNodata=nodata_value
    )
    band_ds = None  # Fermer le dataset après l'opération

In [3]:
reprojected_bands = []

for date_idx, date in enumerate(band_date, start=1):  # Itérer sur chaque date
    for band_idx, band_number in enumerate(band_numbers, start=1):  # Itérer sur chaque bande
        band_file = f"{band_path}/SENTINEL2B{date}{band_number}.tif"  # Générer le chemin complet
        reprojected_file = os.path.join(out_path, f"date_{date_idx}_band_{band_number}_reprojected.tif")  # Fichier de sortie
        reproject_and_resample(band_file, reprojected_file, pixel_size, projection)  # Reprojection
        reprojected_bands.append(reprojected_file)  # Ajouter au suivi des fichiers traités


/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [4]:
# Étape 2 : Reprojeter et rééchantillonner le masque forêt pour correspondre aux dimensions des bandes
def reproject_and_resample_mask(mask_file, output_file, pixel_size, projection):
    mask_ds = gdal.Open(mask_file)
    if mask_ds is None:
        raise Exception(f"Erreur lors de l'ouverture du fichier {mask_file}")
    
    # Reprojection et rééchantillonnage du masque
    gdal.Warp(
        output_file,
        mask_ds,
        xRes=pixel_size,
        yRes=pixel_size,
        dstSRS=projection,
        dstNodata=nodata_value
    )
    mask_ds = None  # Fermer le dataset après l'opération

In [5]:
# Reprojeter et rééchantillonner le masque
reprojected_mask_file = os.path.join(out_path, "masque_foret_reprojected.tif")
reproject_and_resample_mask(mask_file, reprojected_mask_file, pixel_size, projection)
mask_file = reprojected_mask_file  # Remplacer par le masque reprojeté

In [6]:
# Étape 3 : Appliquer le masque forêt à chaque bande
masked_bands = []
for i, band_file in enumerate(reprojected_bands):
    masked_file = os.path.join(out_path, f"band_{i+1}_masked.tif")
    gdal_calc_command = (
        f"gdal_calc.py -A {band_file} -B {mask_file} --outfile={masked_file} "
        f"--calc=\"A*B\" --NoDataValue={nodata_value} --quiet"
    )
    os.system(gdal_calc_command)
    masked_bands.append(masked_file)

In [7]:
# Étape 3 : Fusionner toutes les bandes dans un seul fichier multidimensionnel
merge_command = (
    f"gdal_merge.py -separate -o {output_file} " + " ".join(masked_bands) + f" -a_nodata {nodata_value} -ot {out_type}"
)
os.system(merge_command)


0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [10]:
# Listes des fichiers intermédiaires à supprimer
intermediate_files = reprojected_bands + [reprojected_mask_file] + masked_bands

# Supprimer les fichiers intermédiaires
for file_path in intermediate_files:
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Fichier supprimé : {file_path}")
    else:
        print(f"Fichier introuvable (non supprimé) : {file_path}")

Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_2_reprojected.tif
Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_3_reprojected.tif
Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_4_reprojected.tif
Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_5_reprojected.tif
Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_6_reprojected.tif
Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_7_reprojected.tif
Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_8_reprojected.tif
Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_8A_reprojected.tif
Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_11_reprojected.tif
Fichier introuvable (non supprimé) : ../results/data/img_pretraitees/date_1_band_12_reprojected.tif
Fichier